In [1]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as la
## Construct LSTM using Tensorflow + Keras
# Import Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras import optimizers


######################################################################
######################################################################
######################################################################
## CSP File from: https://github.com/spolsley/common-spatial-patterns

# CSP takes any number of arguments, but each argument must be a collection of trials associated with a task
# That is, for N tasks, N arrays are passed to CSP each with dimensionality (# of trials of task N) x (feature vector)
# Trials may be of any dimension, provided that each trial for each task has the same dimensionality,
# otherwise there can be no spatial filtering since the trials cannot be compared
def CSP(*tasks):
	if len(tasks) < 2:
		print("Must have at least 2 tasks for filtering.")
		return (None,) * len(tasks)
	else:
		filters = ()
		# CSP algorithm
		# For each task x, find the mean variances Rx and not_Rx, which will be used to compute spatial filter SFx
		iterator = range(0,len(tasks))
		for x in iterator:
			# Find Rx
			Rx = covarianceMatrix(tasks[x][0])
			for t in range(1,len(tasks[x])):
				Rx += covarianceMatrix(tasks[x][t])
			Rx = Rx / len(tasks[x])

			# Find not_Rx
			count = 0
			not_Rx = Rx * 0
			for not_x in [element for element in iterator if element != x]:
				for t in range(0,len(tasks[not_x])):
					not_Rx += covarianceMatrix(tasks[not_x][t])
					count += 1
			not_Rx = not_Rx / count

			# Find the spatial filter SFx
			SFx = spatialFilter(Rx,not_Rx)
			filters += (SFx,)

			# Special case: only two tasks, no need to compute any more mean variances
			if len(tasks) == 2:
				filters += (spatialFilter(not_Rx,Rx),)
				break
		return filters

# covarianceMatrix takes a matrix A and returns the covariance matrix, scaled by the variance
def covarianceMatrix(A):
	Ca = np.dot(A,np.transpose(A))/np.trace(np.dot(A,np.transpose(A)))
	return Ca

# spatialFilter returns the spatial filter SFa for mean covariance matrices Ra and Rb
def spatialFilter(Ra,Rb):
	R = Ra + Rb
	E,U = la.eig(R)

	# CSP requires the eigenvalues E and eigenvector U be sorted in descending order
	ord = np.argsort(E)
	ord = ord[::-1] # argsort gives ascending order, flip to get descending
	E = E[ord]
	U = U[:,ord]

	# Find the whitening transformation matrix
	P = np.dot(np.sqrt(la.inv(np.diag(E))),np.transpose(U))

	# The mean covariance matrices may now be transformed
	Sa = np.dot(P,np.dot(Ra,np.transpose(P)))
	Sb = np.dot(P,np.dot(Rb,np.transpose(P)))

	# Find and sort the generalized eigenvalues and eigenvector
	E1,U1 = la.eig(Sa,Sb)
	ord1 = np.argsort(E1)
	ord1 = ord1[::-1]
	E1 = E1[ord1]
	U1 = U1[:,ord1]

	# The projection matrix (the spatial filter) may now be obtained
	SFa = np.dot(np.transpose(U1),P)
	return SFa.astype(np.float32)
######################################################################
def GetMinSteps(indeces, data):
	minVal = 9999;
	for index in indeces:
		length = data[index].shape[1];
		if(length < minVal):
			minVal = length;
	return minVal
######################################################################
def GetData(indeces, dataIn, truncateValue):
	dataOut = []
	truncate = True;
	if truncateValue == 0:
		truncate = False;
	for idx in indeces:
		currentData = dataIn[idx]
		if truncate:
			dataOut.append(currentData[:,0:truncateValue])
		else:
			dataOut.append(currentData)
	return np.asarray(dataOut)
######################################################################
######################################################################
######################################################################


enableDropout = True;
dropoutPercentage = 0.3;


fileNames = ['5F-SubjectB-160309-5St-SGLHand-HFREQ.mat','5F-SubjectB-160311-5St-SGLHand-HFREQ.mat',\
             '5F-SubjectC-160429-5St-SGLHand-HFREQ.mat','5F-SubjectE-160321-5St-SGLHand-HFREQ.mat',\
			 '5F-SubjectF-160210-5St-SGLHand-HFREQ.mat','5F-SubjectG-160413-5St-SGLHand-HFREQ.mat',\
			 '5F-SubjectG-160428-5St-SGLHand-HFREQ.mat','5F-SubjectH-160804-5St-SGLHand-HFREQ.mat',\
			 '5F-SubjectI-160719-5St-SGLHand-HFREQ.mat','5F-SubjectI-160723-5St-SGLHand-HFREQ.mat'];

numDatasets = len(fileNames);

testAccuracyNoCSP = np.zeros((1,numDatasets))
trainAccuracyNoCSP = np.zeros((1,numDatasets))
testAccuracyWithCSP = np.zeros((1,numDatasets))
trainAccuracyWithCSP = np.zeros((1,numDatasets))

for dataset in range(0, numDatasets):

	fileName = fileNames[dataset];

	file = sio.loadmat('../../dataFolder/5F_Data/{}'.format(fileName)) #replace with .mat file name
	header=file['__header__']
	version=file['__version__']
	glob=file['__globals__']
	#ans=file['ans']


	#x=file['x']
	o=file['o'][0][0]
	data=o['data']
	data = np.transpose(data)
	data = data[0:21,:];
	print(data)
	nS=o['nS'][0][0]
	#values of structure seem to be 2D numpy arrays, if originally a scalar in Matlab.
	#use [0][0] to get scalar.
	#print("Number of samples: {numSamples}".format(numSamples=nS))
	test=o['id'][0] #id value became a 1D array of size 1 for some reason. use [0] to get value
	#print("Dataset ID: {id}".format(id=test))
	chnames=o['chnames'][:,0] #[:,0] converts from 2D array back to 1D array
	#print("Channel names: {channelNames}".format(channelNames=chnames))
	markers = o['marker']
	## The markers are all still individual arrays of size 1x1, so we convert them to an array with single values
	markersArray = []
	for marker in markers:
		markersArray.append(marker[0])
	markersArray = np.asarray(markersArray)

	#################################
	#################################
	#5F interaction paradigm
	#1-thumb MI, 2-index finger MI, 3-middle finger MI, 4-ring finger MI, 5-pinkie finger MI

	#all paradigms
	#99-initial relaxation period
	#91-inter-session rest break period
	#92-experiment end
	#################################

	## Find the starting indeces where the marker changes
	changeIdxs = np.where(np.transpose(markersArray)[:-1] != np.transpose(markersArray)[1:])[0]
	#print("Number of index changes: {idxChanges}".format(idxChanges=changeIdxs.shape[0]))
	## Split the data so that it has its matching marker
	dataSplit = np.array_split(data, changeIdxs[:-1], axis=1)
	splitCount = 0
	for splitData in dataSplit:
		splitCount += 1
	#print("Number of arrays in data split: {num}".format(num=splitCount))
	## Retrieve the marker values for each of the change indeces (changeIdxs)
	markerTargets = markersArray[changeIdxs];
	#print("Number of marker targets: {numTargets}".format(numTargets=markerTargets.shape[0]))

	## To Apply CSP, we first only get the indeces for MI tasks 1 and 2 (left and right hand, respectively.)
	tIdx = np.where(markerTargets == 1)[0]
	iIdx = np.where(markerTargets == 2)[0]
	mIdx = np.where(markerTargets == 3)[0]
	rIdx = np.where(markerTargets == 4)[0]
	pIdx = np.where(markerTargets == 5)[0]
	#tCount = tIdx.shape
	#print("Thumb Marker Count: {}\tSize of First: ({},{})".format(tCount, dataSplit[tIdx[0]].shape[0],dataSplit[tIdx[0]].shape[1]))
	#print("Thumb Marker Count: {}\tSize of Second: ({},{})".format(tCount, dataSplit[tIdx[1]].shape[0],dataSplit[tIdx[1]].shape[1]))
	#print("Thumb Marker Count: {}\tSize of Third: ({},{})".format(tCount, dataSplit[tIdx[2]].shape[0],dataSplit[tIdx[2]].shape[1]))


	tIdxMin = GetMinSteps(tIdx, dataSplit)
	iIdxMin = GetMinSteps(iIdx, dataSplit)
	mIdxMin = GetMinSteps(mIdx, dataSplit)
	rIdxMin = GetMinSteps(rIdx, dataSplit)
	pIdxMin = GetMinSteps(pIdx, dataSplit)
	minValues = [tIdxMin, iIdxMin, mIdxMin, rIdxMin, pIdxMin]
	#minValues

	#Truncate the data to the min size
	minValue = np.min(minValues)
	#print(minValue)

	tData = GetData(tIdx, dataSplit, minValue)
	iData = GetData(iIdx, dataSplit, minValue)
	mData = GetData(mIdx, dataSplit, minValue)
	rData = GetData(rIdx, dataSplit, minValue)
	pData = GetData(pIdx, dataSplit, minValue)
	#print("Length of tData: {}".format(len(tData)))
	#print("Length of iData: {}".format(len(iData)))
	#print("Length of mData: {}".format(len(mData)))
	#print("Length of rData: {}".format(len(rData)))
	#print("Length of pData: {}".format(len(pData)))

	minLen = np.min([len(tData), len(iData), len(mData), len(rData), len(pData)])

	##Want to make sure they are balanced, so we keep the minLen values
	tData = tData[0:minLen]
	iData = iData[0:minLen]
	mData = mData[0:minLen]
	rData = rData[0:minLen]
	pData = pData[0:minLen]


	# In[68]:


	#Construct the target arrays and merge the data
	tTargets = np.tile(np.array([1,0,0,0,0]),(minLen,1))
	iTargets = np.tile(np.array([0,1,0,0,0]),(minLen,1))
	mTargets = np.tile(np.array([0,0,1,0,0]),(minLen,1))
	rTargets = np.tile(np.array([0,0,0,1,0]),(minLen,1))
	pTargets = np.tile(np.array([0,0,0,0,1]),(minLen,1))

	markerTargets = np.vstack((tTargets, iTargets, mTargets, rTargets, pTargets))
	FingerData = np.vstack((tData, iData, mData, rData, pData))

	#Sanity Check
	#print("FingerData Shape: {arg1}\tmarkerTargets Shape: {arg2}".format(arg1=FingerData.shape, arg2=markerTargets.shape))

	## Shuffle the data
	FingerData, markerTargets = shuffle(FingerData, markerTargets, random_state=0)

	## Split into train and test sets
	FingerDataTrain, FingerDataTest, markerTargetsTrain, markerTargetsTest = train_test_split(FingerData, markerTargets, test_size=0.3, random_state=1)
	markerTargetsTrain.shape

	## Reshape the data for time-series processing
	## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
	FingerDataTrainRe = FingerDataTrain.reshape((FingerDataTrain.shape[0], FingerDataTrain.shape[1], FingerDataTrain.shape[2]))
	FingerDataTestRe = FingerDataTest.reshape((FingerDataTest.shape[0], FingerDataTest.shape[1], FingerDataTest.shape[2]))

	## Construct the model
	LSTM_EEG = Sequential()
	LSTM_EEG.add(LSTM((100),batch_input_shape=(None,FingerDataTrainRe.shape[1], FingerDataTrainRe.shape[2]), return_sequences=True))
	if(enableDropout):
		LSTM_EEG.add(Dropout(dropoutPercentage))
	LSTM_EEG.add(LSTM((50), return_sequences=False))
	if(enableDropout):
		LSTM_EEG.add(Dropout(dropoutPercentage))
	LSTM_EEG.add(Dense((5),activation='sigmoid'))

	LSTM_EEG.summary()
	sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
	LSTM_EEG.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])


	# In[79]:


	history = LSTM_EEG.fit(FingerDataTrain, markerTargetsTrain, epochs=30,verbose=2, batch_size=16)


	# In[80]:


	predictionsTest = LSTM_EEG.predict(FingerDataTest)


	# In[81]:


	predictionsTest[predictionsTest>0.5] = 1


	# In[82]:


	predictionsTest[predictionsTest <= 0.5] = 0


	# In[83]:


	comparisonArrayTest = predictionsTest == markerTargetsTest


	# In[85]:


	correctCountTest = 0
	for boolValues in comparisonArrayTest:
		if(boolValues[0] & boolValues[1]):
			correctCountTest += 1
	falseCountTest = FingerDataTest.shape[0] - correctCountTest

	predictionsTrain = LSTM_EEG.predict(FingerDataTrain)
	predictionsTrain[predictionsTrain>0.5] = 1;
	predictionsTrain[predictionsTrain<=0.5] = 0;
	comparisonArrayTrain = predictionsTrain == markerTargetsTrain;

	correctCountTrain = 0
	for boolValues in comparisonArrayTrain:
		if(boolValues[0] & boolValues[1]):
			correctCountTrain += 1
	falseCountTrain = FingerDataTrain.shape[0] - correctCountTrain


	# In[87]:
	trainAcc_noCSP = (correctCountTrain*100/FingerDataTrain.shape[0]);
	testAcc_noCSP = (correctCountTest*100/FingerDataTest.shape[0]);

	#Save these values into the array
	trainAccuracyNoCSP[0,dataset] = trainAcc_noCSP;
	testAccuracyNoCSP[0, dataset] = testAcc_noCSP;

	print("#################################")
	print("#################################")
	print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_noCSP))
	print("#################################")
	print("#################################")
	print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_noCSP))
	print("#################################")
	print("#################################")


	# In[88]:


	## Applying CSP to 5F data
	filters = CSP(tData, iData, mData, rData, pData)


	# In[90]:


	filtersArray = np.asarray(filters)


	# In[91]:


	filtersArray.shape


	# In[92]:


	tData_CSP = np.matmul(np.transpose(filtersArray[0]), tData)
	iData_CSP = np.matmul(np.transpose(filtersArray[1]), iData)
	mData_CSP = np.matmul(np.transpose(filtersArray[2]), mData)
	rData_CSP = np.matmul(np.transpose(filtersArray[3]), rData)
	pData_CSP = np.matmul(np.transpose(filtersArray[4]), pData)


	# In[93]:


	FingerData_CSP = np.vstack((tData_CSP, iData_CSP, mData_CSP, rData_CSP, pData_CSP))


	# In[94]:


	#Construct the target arrays and merge the data
	tTargets = np.tile(np.array([1,0,0,0,0]),(minLen,1))
	iTargets = np.tile(np.array([0,1,0,0,0]),(minLen,1))
	mTargets = np.tile(np.array([0,0,1,0,0]),(minLen,1))
	rTargets = np.tile(np.array([0,0,0,1,0]),(minLen,1))
	pTargets = np.tile(np.array([0,0,0,0,1]),(minLen,1))

	markerTargets = np.vstack((tTargets, iTargets, mTargets, rTargets, pTargets))


	# In[95]:


	## Shuffle the data
	FingerData_CSP, markerTargets_CSP = shuffle(FingerData_CSP, markerTargets, random_state=0)
	## Split into train and test sets
	FingerDataTrain_CSP, FingerDataTest_CSP, markerTargetsTrain_CSP, markerTargetsTest_CSP = train_test_split(FingerData_CSP, markerTargets_CSP, test_size=0.3, random_state=1)

	##Use the same model to train and test
	history_CSP = LSTM_EEG.fit(FingerDataTrain_CSP, markerTargetsTrain_CSP, epochs=30,verbose=2, batch_size=16)


	# In[96]:


	predictionsTest = LSTM_EEG.predict(FingerDataTest_CSP)


	# In[97]:


	predictionsTest[predictionsTest>0.5] = 1


	# In[98]:


	predictionsTest[predictionsTest <= 0.5] = 0


	# In[99]:


	comparisonArrayTest = predictionsTest == markerTargetsTest


	# In[100]:


	correctCountTest = 0
	for boolValues in comparisonArrayTest:
		if(boolValues[0] & boolValues[1]):
			correctCountTest += 1
	falseCountTest = FingerDataTest_CSP.shape[0] - correctCountTest

	predictionsTrain = LSTM_EEG.predict(FingerDataTrain_CSP)
	predictionsTrain[predictionsTrain>0.5] = 1;
	predictionsTrain[predictionsTrain<=0.5] = 0;
	comparisonArrayTrain = predictionsTrain == markerTargetsTrain;

	correctCountTrain = 0
	for boolValues in comparisonArrayTrain:
		if(boolValues[0] & boolValues[1]):
			correctCountTrain += 1
	falseCountTrain = FingerDataTrain_CSP.shape[0] - correctCountTrain

	#Computing the accuracy
	trainAcc_wCSP = (correctCountTrain*100/FingerDataTrain_CSP.shape[0]);
	testAcc_wCSP = (correctCountTest*100/FingerDataTest_CSP.shape[0]);

	#Save these values into the array
	trainAccuracyWithCSP[0,dataset] = trainAcc_wCSP;
	testAccuracyWithCSP[0, dataset] = testAcc_wCSP;

	print("#################################")
	print("### RESULTS AFTER APPLYING CSP ##")
	print("#################################")
	print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_wCSP))
	print("#################################")
	print("#################################")
	print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_wCSP))
	print("#################################")
	print("#################################")



[[-0.   -0.   -0.   ... -7.2  -5.94 -0.72]
 [-0.   -0.   -0.   ... -8.96 -7.28 -7.08]
 [-0.   -0.   -0.   ...  9.    9.18  9.3 ]
 ...
 [-0.   -0.   -0.   ... 27.67  8.23  7.75]
 [-0.   -0.   -0.   ... 25.41 26.82 29.07]
 [-0.   -0.   -0.   ... 17.53 16.29 22.08]]
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 21, 100)           550400    
_________________________________________________________________
dropout (Dropout)            (None, 21, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 2

/opt/conda/envs/py37-tf2/lib/python3.7/site-packages/ipykernel_launcher.py:93: ComplexWarning: Casting complex values to real discards the imaginary part


Train on 595 samples
Epoch 1/30
595/595 - 0s - loss: 0.6076 - accuracy: 0.7516
Epoch 2/30
595/595 - 0s - loss: 0.4833 - accuracy: 0.7976
Epoch 3/30
595/595 - 0s - loss: 0.4431 - accuracy: 0.8128
Epoch 4/30
595/595 - 0s - loss: 0.4056 - accuracy: 0.8242
Epoch 5/30
595/595 - 0s - loss: 0.3844 - accuracy: 0.8326
Epoch 6/30
595/595 - 0s - loss: 0.3774 - accuracy: 0.8366
Epoch 7/30
595/595 - 0s - loss: 0.3599 - accuracy: 0.8434
Epoch 8/30
595/595 - 0s - loss: 0.3394 - accuracy: 0.8568
Epoch 9/30
595/595 - 0s - loss: 0.3215 - accuracy: 0.8608
Epoch 10/30
595/595 - 0s - loss: 0.3010 - accuracy: 0.8760
Epoch 11/30
595/595 - 0s - loss: 0.2867 - accuracy: 0.8739
Epoch 12/30
595/595 - 0s - loss: 0.2702 - accuracy: 0.8891
Epoch 13/30
595/595 - 0s - loss: 0.2529 - accuracy: 0.8961
Epoch 14/30
595/595 - 0s - loss: 0.2490 - accuracy: 0.8975
Epoch 15/30
595/595 - 0s - loss: 0.2347 - accuracy: 0.9039
Epoch 16/30
595/595 - 0s - loss: 0.2207 - accuracy: 0.9119
Epoch 17/30
595/595 - 0s - loss: 0.2131 - ac

/opt/conda/envs/py37-tf2/lib/python3.7/site-packages/ipykernel_launcher.py:93: ComplexWarning: Casting complex values to real discards the imaginary part


Train on 605 samples
Epoch 1/30
605/605 - 0s - loss: 0.6177 - accuracy: 0.7471
Epoch 2/30
605/605 - 0s - loss: 0.4840 - accuracy: 0.7858
Epoch 3/30
605/605 - 0s - loss: 0.4335 - accuracy: 0.8093
Epoch 4/30
605/605 - 0s - loss: 0.4218 - accuracy: 0.8139
Epoch 5/30
605/605 - 0s - loss: 0.3944 - accuracy: 0.8331
Epoch 6/30
605/605 - 0s - loss: 0.3839 - accuracy: 0.8340
Epoch 7/30
605/605 - 0s - loss: 0.3685 - accuracy: 0.8407
Epoch 8/30
605/605 - 0s - loss: 0.3283 - accuracy: 0.8641
Epoch 9/30
605/605 - 0s - loss: 0.3167 - accuracy: 0.8664
Epoch 10/30
605/605 - 0s - loss: 0.2897 - accuracy: 0.8826
Epoch 11/30
605/605 - 0s - loss: 0.2839 - accuracy: 0.8813
Epoch 12/30
605/605 - 0s - loss: 0.2490 - accuracy: 0.9028
Epoch 13/30
605/605 - 0s - loss: 0.2363 - accuracy: 0.9048
Epoch 14/30
605/605 - 0s - loss: 0.2376 - accuracy: 0.9061
Epoch 15/30
605/605 - 0s - loss: 0.2466 - accuracy: 0.8995
Epoch 16/30
605/605 - 0s - loss: 0.2197 - accuracy: 0.9098
Epoch 17/30
605/605 - 0s - loss: 0.2199 - ac

...

Train on 584 samples
Epoch 1/30
584/584 - 0s - loss: 0.5502 - accuracy: 0.7640
Epoch 2/30
584/584 - 0s - loss: 0.5097 - accuracy: 0.7959
Epoch 3/30
584/584 - 0s - loss: 0.4891 - accuracy: 0.8024
Epoch 4/30
584/584 - 0s - loss: 0.4799 - accuracy: 0.8017
Epoch 5/30
584/584 - 0s - loss: 0.4792 - accuracy: 0.7990
Epoch 6/30
584/584 - 0s - loss: 0.4705 - accuracy: 0.8003
Epoch 7/30
584/584 - 0s - loss: 0.4695 - accuracy: 0.8058
Epoch 8/30
584/584 - 0s - loss: 0.4664 - accuracy: 0.8041
Epoch 9/30
584/584 - 0s - loss: 0.4613 - accuracy: 0.8003
Epoch 10/30
584/584 - 0s - loss: 0.4585 - accuracy: 0.8065
Epoch 11/30
584/584 - 0s - loss: 0.4532 - accuracy: 0.8110
Epoch 12/30
584/584 - 0s - loss: 0.4523 - accuracy: 0.8096
Epoch 13/30
584/584 - 0s - loss: 0.4474 - accuracy: 0.8106
Epoch 14/30
584/584 - 0s - loss: 0.4443 - accuracy: 0.8058
Epoch 15/30
584/584 - 0s - loss: 0.4350 - accuracy: 0.8103
Epoch 16/30
584/584 - 0s - loss: 0.4320 - accuracy: 0.8144
Epoch 17/30
584/584 - 0s - loss: 0.4404 - ac

/opt/conda/envs/py37-tf2/lib/python3.7/site-packages/ipykernel_launcher.py:93: ComplexWarning: Casting complex values to real discards the imaginary part


Train on 602 samples
Epoch 1/30
602/602 - 0s - loss: 0.5714 - accuracy: 0.7631
Epoch 2/30
602/602 - 0s - loss: 0.4807 - accuracy: 0.7940
Epoch 3/30
602/602 - 0s - loss: 0.4578 - accuracy: 0.8086
Epoch 4/30
602/602 - 0s - loss: 0.4485 - accuracy: 0.8050
Epoch 5/30
602/602 - 0s - loss: 0.4288 - accuracy: 0.8173
Epoch 6/30
602/602 - 0s - loss: 0.4051 - accuracy: 0.8269
Epoch 7/30
602/602 - 0s - loss: 0.4043 - accuracy: 0.8226
Epoch 8/30
602/602 - 0s - loss: 0.4010 - accuracy: 0.8286
Epoch 9/30
602/602 - 0s - loss: 0.3823 - accuracy: 0.8375
Epoch 10/30
602/602 - 0s - loss: 0.3830 - accuracy: 0.8365
Epoch 11/30
602/602 - 0s - loss: 0.3825 - accuracy: 0.8382
Epoch 12/30
602/602 - 0s - loss: 0.3712 - accuracy: 0.8472
Epoch 13/30
602/602 - 0s - loss: 0.3584 - accuracy: 0.8512
Epoch 14/30
602/602 - 0s - loss: 0.3559 - accuracy: 0.8495
Epoch 15/30
602/602 - 0s - loss: 0.3364 - accuracy: 0.8608
Epoch 16/30
602/602 - 0s - loss: 0.3320 - accuracy: 0.8638
Epoch 17/30
602/602 - 0s - loss: 0.3365 - ac

/opt/conda/envs/py37-tf2/lib/python3.7/site-packages/ipykernel_launcher.py:93: ComplexWarning: Casting complex values to real discards the imaginary part


Train on 605 samples
Epoch 1/30
605/605 - 0s - loss: 0.5445 - accuracy: 0.7755
Epoch 2/30
605/605 - 0s - loss: 0.4891 - accuracy: 0.7990
Epoch 3/30
605/605 - 0s - loss: 0.4720 - accuracy: 0.8036
Epoch 4/30
605/605 - 0s - loss: 0.4677 - accuracy: 0.8033
Epoch 5/30
605/605 - 0s - loss: 0.4555 - accuracy: 0.8089
Epoch 6/30
605/605 - 0s - loss: 0.4525 - accuracy: 0.8083
Epoch 7/30
605/605 - 0s - loss: 0.4399 - accuracy: 0.8109
Epoch 8/30
605/605 - 0s - loss: 0.4287 - accuracy: 0.8159
Epoch 9/30
605/605 - 0s - loss: 0.4218 - accuracy: 0.8188
Epoch 10/30
605/605 - 0s - loss: 0.4152 - accuracy: 0.8165
Epoch 11/30
605/605 - 0s - loss: 0.4100 - accuracy: 0.8205
Epoch 12/30
605/605 - 0s - loss: 0.4051 - accuracy: 0.8284
Epoch 13/30
605/605 - 0s - loss: 0.4011 - accuracy: 0.8284
Epoch 14/30
605/605 - 0s - loss: 0.4027 - accuracy: 0.8301
Epoch 15/30
605/605 - 0s - loss: 0.3936 - accuracy: 0.8281
Epoch 16/30
605/605 - 0s - loss: 0.3968 - accuracy: 0.8334
Epoch 17/30
605/605 - 0s - loss: 0.3847 - ac

In [2]:
fileNames

['5F-SubjectB-160309-5St-SGLHand-HFREQ.mat',
 '5F-SubjectB-160311-5St-SGLHand-HFREQ.mat',
 '5F-SubjectC-160429-5St-SGLHand-HFREQ.mat',
 '5F-SubjectE-160321-5St-SGLHand-HFREQ.mat',
 '5F-SubjectF-160210-5St-SGLHand-HFREQ.mat',
 '5F-SubjectG-160413-5St-SGLHand-HFREQ.mat',
 '5F-SubjectG-160428-5St-SGLHand-HFREQ.mat',
 '5F-SubjectH-160804-5St-SGLHand-HFREQ.mat',
 '5F-SubjectI-160719-5St-SGLHand-HFREQ.mat',
 '5F-SubjectI-160723-5St-SGLHand-HFREQ.mat']

In [3]:
testAccuracyNoCSP

array([[50.19607843, 53.07692308, 55.29411765, 46.15384615, 40.76923077,
        61.02362205, 51.53846154, 52.19123506, 51.5503876 , 58.84615385]])

In [4]:
testAccuracyWithCSP

array([[85.49019608, 79.61538462, 85.09803922, 66.53846154, 61.15384615,
        90.5511811 , 89.61538462, 58.96414343, 68.60465116, 73.84615385]])

In [5]:
trainAccuracyNoCSP

array([[97.14285714, 94.7107438 , 96.63865546, 94.7107438 , 97.68595041,
        86.12521151, 88.09917355, 85.95890411, 91.86046512, 81.15702479]])

In [6]:
trainAccuracyWithCSP

array([[98.48739496, 91.90082645, 97.14285714, 87.27272727, 92.56198347,
        96.27749577, 98.34710744, 74.14383562, 87.20930233, 81.65289256]])

In [7]:
combinedArray = np.transpose(np.asarray(np.vstack((trainAccuracyNoCSP,testAccuracyNoCSP,trainAccuracyWithCSP, testAccuracyWithCSP))))

In [8]:
combinedArray.shape

(10, 4)

In [11]:
np.savetxt('5FperSubject_Dropout.csv', combinedArray, delimiter=',')

In [10]:
pwd

'/home/davalenc/ProjectFiles/ECE228ECGProject/JupyterNotebooks'